<a href="https://colab.research.google.com/github/ContiPaolo/Multifidelity-Tutorial/blob/main/MF_POD_Burger's1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-fidelity reduced-order modeling on Groundwater's flow - Darcy equation**

#### We aim to create a reduced-order surrogate model that enables the efficient recovery of high-fidelities solutions parameter variations, starting from their low-fidelity counterparts.

#### We consider a data-driven strategy that combines a coarser FEM solver with multi-fidelity neural network surrogates.


#### Import libraries

In [1]:
import numpy as np
#import matplotlib as plt
from matplotlib import pyplot as plt
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.optimizers import Adam,Nadam,Adamax
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from sklearn.utils import extmath
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import KFold


#######################     CONFIGURATIONS     ##########################
seed = 29
train = True
save = True

# DATASET 
The dataset has been developed using fenics library, it is composed of 64000 samples divided in 90% training and 10% testing.\
The input (or X) are the 64 first component of the Karhunen-Loève (KL) decompositions. \
These eigenmodes allow to paratetrize a random field in the most accurate way.

The output: 25 sensors are used to record the hydraulic pressure in the domain $\Omega$ \
The sensors are distributed on a grid with positions [0.1, 0.3, 0.5, 0.7, 0.9]



Training Multi Fidelity connected neural network surrogate model

In [3]:
import numpy as np
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Add
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

Training = True

coeff = 1e-6
n_neurons = 256
n_splits = 4
n_sample = 1000

model_lf = load_model(f'./models/model_25resolution_64000samples_1.keras')

# Load the data
X_train_params = np.loadtxt('../../data/50-25-10/X_train_50resolution.csv', delimiter=',')[:n_sample,:]
X_train_pod = model_lf(X_train_params).numpy()[:n_sample,:]
y_train = np.loadtxt('../../data/50-25-10/y_train_50resolution.csv', delimiter=',')[:n_sample,:]

print(np.mean((X_train_pod - y_train)**2))

print(y_train)

X_test_params = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter=',')
X_test_pod = model_lf(X_test_params).numpy()
y_test = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv', delimiter=',')

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch > 1000:
        return lr*(epoch)/(1+epoch)
    else:
        return max(lr * 0.999, 1e-7)
    
# Setting up K-Fold Cross-Validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Cross-validation loop
fold_var = 1

# Define the three branches of the model
input_params = Input(shape=(X_train_params.shape[1],))
input_pod = Input(shape=(X_train_pod.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_params)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(combined)
output = Dense(25, activation='linear', kernel_regularizer=l2(coeff))(combined)

# Create the model
model = Model(inputs=[input_params,input_pod], outputs=output)

for train_index, val_index in kf.split(X_train_params):

    print(f"Training fold {fold_var}...")

    # Splitting the data into training and validation sets
    X_train_params_k, X_val_params_k = X_train_params[train_index], X_train_params[val_index]
    X_train_pod_k, X_val_pod_k = X_train_pod[train_index], X_train_pod[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]

    # Compile the model
    initial_learning_rate = 0.001
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

    # Define the learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(scheduler)

    # Train the model
    history = model.fit([X_train_params_k, X_train_pod_k], y_train_k,
                        epochs=1000,
                        batch_size=64,
                        validation_data=([X_val_params_k, X_val_pod_k], y_val_k),
                        validation_freq=10,
                        callbacks=[lr_scheduler])
    
    # Save the model for each fold
    model.save(f'./models/model_2step_50-25resolution_{n_sample}samples_{fold_var}_coeff{coeff}.keras')

    # Increment the fold number
    fold_var += 1




0.00029798808389350483
[[0.82843896 0.80762808 0.91036802 ... 0.19904088 0.23674059 0.32839533]
 [0.8851561  0.89120142 0.84836334 ... 0.04889952 0.05757054 0.06519809]
 [0.93718216 0.89983595 0.93218052 ... 0.06193638 0.03079713 0.01388359]
 ...
 [0.91650792 0.95266163 0.95655705 ... 0.07533477 0.09901553 0.10143102]
 [0.90753436 0.91862464 0.93477205 ... 0.06207423 0.03272273 0.10606732]
 [0.92125208 0.90308038 0.92070229 ... 0.04296381 0.01238305 0.00600186]]
Training fold 1...
Epoch 1/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2011 - mean_squared_error: 0.1993 - learning_rate: 0.0010
Epoch 2/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0131 - mean_squared_error: 0.0114 - learning_rate: 0.0010
Epoch 3/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0080 - mean_squared_error: 0.0063 - learning_rate: 0.0010
Epoch 4/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0064 - mean_squared_error: 0.0046 - learning_rate: 0.0010
Epoch 5/1000
12/12 ━━━━━━━━━━━━━━━

In [5]:
import numpy as np
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Add
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

Training = True

coeff = 1e-7
n_neurons = 256
n_splits = 4
n_sample = 2000

model_lf = load_model(f'./models/model_10resolution_64000samples_1.keras')

# Load the data
X_train_params = np.loadtxt('../../data/50-25-10/X_train_50resolution.csv', delimiter=',')[:n_sample,:]
X_train_pod = model_lf(X_train_params).numpy()[:n_sample,:]
y_train = np.loadtxt('../../data/50-25-10/y_train_50resolution.csv', delimiter=',')[:n_sample,:]

print(np.mean((X_train_pod - y_train)**2))

print(y_train)

X_test_params = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter=',')
X_test_pod = model_lf(X_test_params).numpy()
y_test = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv', delimiter=',')

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch > 1000:
        return lr*(epoch)/(1+epoch)
    else:
        return max(lr * 0.999, 1e-7)
    
# Setting up K-Fold Cross-Validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Cross-validation loop
fold_var = 1

# Define the three branches of the model
input_params = Input(shape=(X_train_params.shape[1],))
input_pod = Input(shape=(X_train_pod.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_params)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(combined)
output = Dense(25, activation='linear', kernel_regularizer=l2(coeff))(combined)

# Create the model
model = Model(inputs=[input_params,input_pod], outputs=output)

for train_index, val_index in kf.split(X_train_params):

    print(f"Training fold {fold_var}...")

    # Splitting the data into training and validation sets
    X_train_params_k, X_val_params_k = X_train_params[train_index], X_train_params[val_index]
    X_train_pod_k, X_val_pod_k = X_train_pod[train_index], X_train_pod[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]

    # Compile the model
    initial_learning_rate = 0.001
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

    # Define the learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(scheduler)

    # Train the model
    history = model.fit([X_train_params_k, X_train_pod_k], y_train_k,
                        epochs=1000,
                        batch_size=64,
                        validation_data=([X_val_params_k, X_val_pod_k], y_val_k),
                        validation_freq=10,
                        callbacks=[lr_scheduler])
    
    # Save the model for each fold
    model.save(f'./models/model_2step_50-10resolution_{n_sample}samples_{fold_var}_coeff{coeff}.keras')

    # Increment the fold number
    fold_var += 1




0.0002916355300577195
[[0.82843896 0.80762808 0.91036802 ... 0.19904088 0.23674059 0.32839533]
 [0.8851561  0.89120142 0.84836334 ... 0.04889952 0.05757054 0.06519809]
 [0.93718216 0.89983595 0.93218052 ... 0.06193638 0.03079713 0.01388359]
 ...
 [0.85917021 0.86226574 0.86832887 ... 0.0499593  0.04998812 0.06185904]
 [0.92967734 0.91817288 0.92903502 ... 0.14890726 0.0953635  0.07168957]
 [0.96514926 0.97976565 0.93927461 ... 0.15908284 0.10043792 0.19039906]]
Training fold 1...
Epoch 1/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1241 - mean_squared_error: 0.1239 - learning_rate: 0.0010
Epoch 2/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0060 - mean_squared_error: 0.0058 - learning_rate: 0.0010
Epoch 3/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0034 - mean_squared_error: 0.0033 - learning_rate: 0.0010
Epoch 4/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0024 - mean_squared_error: 0.0022 - learning_rate: 0.0010
Epoch 5/1000
24/24 ━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Add
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

Training = True

coeff = 1e-7
n_neurons = 256
n_splits = 4
n_sample = 4000

model_lf = load_model(f'./models/model_10resolution_64000samples_1.keras')

# Load the data
X_train_params = np.loadtxt('../../data/50-25-10/X_train_50resolution.csv', delimiter=',')[:n_sample,:]
X_train_pod = model_lf(X_train_params).numpy()[:n_sample,:]
y_train = np.loadtxt('../../data/50-25-10/y_train_50resolution.csv', delimiter=',')[:n_sample,:]

print(np.mean((X_train_pod - y_train)**2))

print(y_train)

X_test_params = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter=',')
X_test_pod = model_lf(X_test_params).numpy()
y_test = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv', delimiter=',')

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch > 1000:
        return lr*(epoch)/(1+epoch)
    else:
        return max(lr * 0.999, 1e-7)
    
# Setting up K-Fold Cross-Validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Cross-validation loop
fold_var = 1

# Define the three branches of the model
input_params = Input(shape=(X_train_params.shape[1],))
input_pod = Input(shape=(X_train_pod.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_params)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(combined)
output = Dense(25, activation='linear', kernel_regularizer=l2(coeff))(combined)

# Create the model
model = Model(inputs=[input_params,input_pod], outputs=output)

for train_index, val_index in kf.split(X_train_params):

    print(f"Training fold {fold_var}...")

    # Splitting the data into training and validation sets
    X_train_params_k, X_val_params_k = X_train_params[train_index], X_train_params[val_index]
    X_train_pod_k, X_val_pod_k = X_train_pod[train_index], X_train_pod[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]

    # Compile the model
    initial_learning_rate = 0.001
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

    # Define the learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(scheduler)

    # Train the model
    history = model.fit([X_train_params_k, X_train_pod_k], y_train_k,
                        epochs=1000,
                        batch_size=64,
                        validation_data=([X_val_params_k, X_val_pod_k], y_val_k),
                        validation_freq=10,
                        callbacks=[lr_scheduler])
    
    # Save the model for each fold
    model.save(f'./models/model_2step_50-10resolution_{n_sample}samples_{fold_var}_coeff{coeff}.keras')

    # Increment the fold number
    fold_var += 1




In [ ]:
import numpy as np
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Add
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

Training = True

coeff = 1e-8
n_neurons = 256
n_splits = 4
n_sample = 8000

model_lf = load_model(f'./models/model_10resolution_64000samples_1.keras')

# Load the data
X_train_params = np.loadtxt('../../data/50-25-10/X_train_50resolution.csv', delimiter=',')[:n_sample,:]
X_train_pod = model_lf(X_train_params).numpy()[:n_sample,:]
y_train = np.loadtxt('../../data/50-25-10/y_train_50resolution.csv', delimiter=',')[:n_sample,:]

print(np.mean((X_train_pod - y_train)**2))

print(y_train)

X_test_params = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter=',')
X_test_pod = model_lf(X_test_params).numpy()
y_test = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv', delimiter=',')

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch > 1000:
        return lr*(epoch)/(1+epoch)
    else:
        return max(lr * 0.99, 1e-7)
    
# Setting up K-Fold Cross-Validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Cross-validation loop
fold_var = 1

# Define the three branches of the model
input_params = Input(shape=(X_train_params.shape[1],))
input_pod = Input(shape=(X_train_pod.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_params)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(combined)
output = Dense(25, activation='linear', kernel_regularizer=l2(coeff))(combined)

# Create the model
model = Model(inputs=[input_params,input_pod], outputs=output)

for train_index, val_index in kf.split(X_train_params):

    print(f"Training fold {fold_var}...")

    # Splitting the data into training and validation sets
    X_train_params_k, X_val_params_k = X_train_params[train_index], X_train_params[val_index]
    X_train_pod_k, X_val_pod_k = X_train_pod[train_index], X_train_pod[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]

    # Compile the model
    initial_learning_rate = 0.001
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

    # Define the learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(scheduler)

    # Train the model
    history = model.fit([X_train_params_k, X_train_pod_k], y_train_k,
                        epochs=1000,
                        batch_size=64,
                        validation_data=([X_val_params_k, X_val_pod_k], y_val_k),
                        validation_freq=10,
                        callbacks=[lr_scheduler])
    
    # Save the model for each fold
    model.save(f'./models/model_2step_50-10resolution_{n_sample}samples_{fold_var}_coeff{coeff}.keras')

    # Increment the fold number
    fold_var += 1




In [ ]:
import numpy as np
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Add
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

Training = True

coeff = 1e-8
n_neurons = 256
n_splits = 4
n_sample = 16000

model_lf = load_model(f'./models/model_10resolution_64000samples_1.keras')

# Load the data
X_train_params = np.loadtxt('../../data/50-25-10/X_train_50resolution.csv', delimiter=',')[:n_sample,:]
X_train_pod = model_lf(X_train_params).numpy()[:n_sample,:]
y_train = np.loadtxt('../../data/50-25-10/y_train_50resolution.csv', delimiter=',')[:n_sample,:]

print(np.mean((X_train_pod - y_train)**2))

print(y_train)

X_test_params = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter=',')
X_test_pod = model_lf(X_test_params).numpy()
y_test = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv', delimiter=',')

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch > 1000:
        return lr*(epoch)/(1+epoch)
    else:
        return max(lr * 0.99, 1e-7)
    
# Setting up K-Fold Cross-Validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Cross-validation loop
fold_var = 1

# Define the three branches of the model
input_params = Input(shape=(X_train_params.shape[1],))
input_pod = Input(shape=(X_train_pod.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_params)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)
x1 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(coeff))(combined)
output = Dense(25, activation='linear', kernel_regularizer=l2(coeff))(combined)

# Create the model
model = Model(inputs=[input_params,input_pod], outputs=output)

for train_index, val_index in kf.split(X_train_params):

    print(f"Training fold {fold_var}...")

    # Splitting the data into training and validation sets
    X_train_params_k, X_val_params_k = X_train_params[train_index], X_train_params[val_index]
    X_train_pod_k, X_val_pod_k = X_train_pod[train_index], X_train_pod[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]

    # Compile the model
    initial_learning_rate = 0.001
    optimizer = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

    # Define the learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(scheduler)

    # Train the model
    history = model.fit([X_train_params_k, X_train_pod_k], y_train_k,
                        epochs=1000,
                        batch_size=64,
                        validation_data=([X_val_params_k, X_val_pod_k], y_val_k),
                        validation_freq=10,
                        callbacks=[lr_scheduler])
    
    # Save the model for each fold
    model.save(f'./models/model_2step_50-10resolution_{n_sample}samples_{fold_var}_coeff{coeff}.keras')

    # Increment the fold number
    fold_var += 1




In [35]:
model = load_model(f'./models/model_50-10resolution_{n_sample}samples_4.keras')
model((X_test_params,X_test_pod)).numpy()

array([[0.9412804 , 0.96021557, 0.9310102 , ..., 0.11300899, 0.10804499,
        0.08837804],
       [0.9211987 , 0.9238984 , 0.9598889 , ..., 0.06384155, 0.06865404,
        0.08434081],
       [0.9307978 , 0.9188586 , 0.8705659 , ..., 0.07316414, 0.15586762,
        0.1932146 ],
       ...,
       [0.9303663 , 0.8583942 , 0.8404265 , ..., 0.14173916, 0.07137661,
        0.06788851],
       [0.95010877, 0.92889416, 0.9032178 , ..., 0.09457538, 0.10062466,
        0.08408783],
       [0.8260754 , 0.74899834, 0.85366315, ..., 0.1172493 , 0.10054297,
        0.10147938]], dtype=float32)